In [1]:
import csv
from datetime import datetime
import json
from operator import itemgetter

MAIN_MENU = """\nPlease choose from the options below
 1. Load trading data
 2. Load current stock prices
 3. Manually enter a new trade
 4. View trading data
 5. View current portfolio
 6. Save trading data
 7. Quit"""


def main():
    trading_data = []
    current_prices = {}

    print("Welcome to the Trader Assistant")
    print("Programmed by David Thompson\n")

    print(MAIN_MENU)
    menu_choice = input(">>> ")
    while menu_choice != "7":
        if menu_choice == "1":
            trading_data.extend(load_trading_data())
        elif menu_choice == "2":
            current_prices = load_current_stock_prices()
        elif menu_choice == "3":
            trading_data.append(manually_enter_trade())
        elif menu_choice == "4":
            if check_trades_exist(trading_data):
                view_trading_data(filter_trading_data(trading_data))
        elif menu_choice == "5":
            if check_trades_exist(trading_data):
                view_current_portfolio(calculate_portfolio_value(trading_data,
                                                                 current_prices))
        elif menu_choice == "6":
            if check_trades_exist(trading_data):
                save_trading_data(trading_data)
        else:
            print("Invalid option!")
        print(MAIN_MENU)
        menu_choice = input(">>> ")
    print("Trading Assistant has ended. Goodbye.")

    
def check_trades_exist(trading_data):
    if len(trading_data) > 0:
        return True
    else:
        print("No trades found. Load or manually enter trades.")
        return False
    
    
# Error messages if file format is invalid.
def load_trading_data():
    trading_data = []
    
    valid_file_format = False
    while not valid_file_format:
        file_in = get_file("Enter a file (.csv): ", "csv")   
        try:
            csv_reader = csv.reader(file_in)
            for row in csv_reader:
                row[2] = int(row[2])
                row[3] = float(row[3])
                row[4] = datetime.strptime(row[4], "%Y-%m-%d").date()
                trading_data.append(row)
            file_in.close()
            valid_file_format = True
        except ValueError:
                print("File format invalid.")
        except IndexError:
                print("File format invalid.")
                
    print(f"{len(trading_data)} trades loaded.")
    return trading_data


# Error messages if file format is invalid.
def load_current_stock_prices():
    current_prices = {}
    
    valid_file_format = False
    while not valid_file_format:
        file_in = get_file("Enter a file (json): ", "json")
        try:
            current_prices = json.load(file_in)
            file_in.close()
            valid_file_format = True
        except json.JSONDecodeError:
            print("File format invalid.")
            
    print(f"{len(current_prices)} current prices loaded.")
    return current_prices


# Checks a file exists.
def get_file(message, file_format):
    valid_file_name = False
    while not valid_file_name:
        file_name = get_not_blank_input(message)
        try:
            if file_format == "csv":
                file_in = open(file_name, "r", newline="") 
            else:
                file_in = open(file_name, "r")
            valid_file_name = True
        except FileNotFoundError:
            print("File not found.")
        except OSError:
            print("File name invalid.")
    return file_in


def get_not_blank_input(message):
    input_name = input(message)
    while input_name == "":
        print("Cannot be blank.")
        input_name = input(message)
    return input_name


def manually_enter_trade():    
    ticker = get_valid_ticker("Enter ticker symbol: ", allow_blank=False)
    buy_or_sell = get_buy_or_sell()
    value = get_valid_float(
        f"{'Enter the cost (including brokerage): ' if buy_or_sell == 'b' else 'Enter sale amount (excluding brokerage): '}")
    quantity = get_valid_integer()
    date = get_valid_date()
    trade = [ticker, buy_or_sell, quantity, value, date]
    view_manual_trade(trade)
    return trade

           
def view_manual_trade(trade):
    ticker = trade[0]
    buy_or_sell = trade[1]
    quantity = trade[2]
    value = trade[3]
    date = trade[4]                      
    print(f"{'Ticker:':<35} {ticker}")
    print(f"{'Buy or sell:':<35} {buy_or_sell}")
    print(f"{'Quantity of stock:':<35} {quantity}")
    print(f"{'Total cost (including brokerage):' if buy_or_sell == 'b' else 'Total proceeds (less brokerage):':<35} {value}")
    print(f"{'Date:':<35} {date}")
    print(f"{date} {ticker:<13}{'BUY' if buy_or_sell == 'b' else 'SELL':<13}{quantity} {'for $':<13}{value}")
    print("Trade added to system.")


def get_valid_ticker(prompt, allow_blank):
    if not allow_blank:
        ticker = get_not_blank_input(prompt)
        while not ticker.isalpha():
            print("Enter letters only.")
            ticker = get_not_blank_input(prompt)
    else:
        ticker = input(prompt)
        while not ticker.isalpha() and not ticker == "":
            print("Enter letters only.")
            ticker = input(prompt)
    return ticker.upper()  


def get_buy_or_sell():
    buy_or_sell = input("Enter buy (b) or sell (s): ")
    while buy_or_sell != "b" and buy_or_sell != "s":
        print("Invalid. Enter the specified options.")
        buy_or_sell = input("Enter buy (b) or sell (s): ")
    return buy_or_sell

    
def get_valid_date():
    valid_input = False
    while not valid_input:
        try:
            date_string = get_not_blank_input("Enter date of trade in yyyy-mm-dd format: ")
            date_object = datetime.strptime(date_string, "%Y-%m-%d").date()
            valid_input = True
        except ValueError:
            print("Enter a date in the valid format.")
    return date_object


def get_valid_float(prompt):
    valid_input = False
    while not valid_input:
        try:
            value = float(get_not_blank_input(prompt))
            valid_input = True
        except ValueError:
            print("Invalid. Enter dollars and cents.")
    return value


def get_valid_integer():
    valid_input = False
    while not valid_input:
        try:
            quantity = int(get_not_blank_input("Enter a quantity traded: "))
            valid_input = True
        except ValueError:
            print("Enter a correct quantity (must be a whole number).")
    return quantity


def view_trading_data(filtered_trading_data):
    trade_types = {"b": "BUY", "s": "SELL"}
    print(f"{len(filtered_trading_data)} trades listed.")
    for trade in filtered_trading_data:
        ticker = trade[0]
        trade_type = trade_types[trade[1]]
        quantity = trade[2]
        dollar_value = trade[3]
        date = trade[4]
        print(f"{date} {ticker:<12}{trade_type:<4}{quantity:>12} for ${dollar_value:>12}")
                              

def filter_trading_data(trading_data):
    filtered_trading_data = []
    ticker_selection = get_valid_ticker(
        "Ticker (leave blank for all): ", allow_blank=True)
    is_reverse_chronological = input("Sort dates in reverse chronological order? (y/n) ")
    
    if ticker_selection == "":
        filtered_trading_data = trading_data
    else:
        for trade in trading_data:
            if ticker_selection == trade[0]:
                filtered_trading_data.append(trade)
                
    if is_reverse_chronological == "y":
        filtered_trading_data.sort(reverse=True, key=itemgetter(4))
    else:
        filtered_trading_data.sort(key=itemgetter(4))
        
    return filtered_trading_data
        
        
def view_current_portfolio(portfolio):
    for key, values in sorted(portfolio.items()):
        ticker = key
        units = values[0]
        value = values[1]
        is_valued = values[2]
        print(ticker)
        print(f"Total units: {units:>25}")
        print(f"Total value: $ {value:>25}" if is_valued else "Current value unknown.")


def calculate_portfolio_value(trading_data, current_prices):
    # Create a portfolio of shares owned or previously owned
    portfolio = {}
    for trade in trading_data:
        ticker = trade[0]
        portfolio[ticker] = []

    # Accumulate quantities and calculate valuations.
    quantity = 0
    for ticker in portfolio.keys():
        for trade in trading_data:
            if ticker == trade[0]:
                if trade[1] == "b":
                    quantity = quantity + trade[2]
                else:
                    quantity = quantity - trade[2]
        if ticker in current_prices:
            value = quantity * current_prices[ticker]
            is_valued = True
        else:
            value = ""
            is_valued = False
        portfolio[ticker] = [quantity, value, is_valued]
        quantity = 0
    return portfolio

        
def save_trading_data(trading_data):
    valid_file_name = False
    while not valid_file_name:
        file_name = get_not_blank_input("Enter a file name: ")
        try:
            file_out = open(file_name, "w", newline="")
            valid_file_name = True
        except OSError:
            print("File name invalid.")
    
    csv_writer = csv.writer(file_out)
    for trade in trading_data:
        csv_writer.writerow(trade)
    print("File saved.")


main()

Welcome to the Trader Assistant
Programmed by David Thompson


Please choose from the options below
 1. Load trading data
 2. Load current stock prices
 3. Manually enter a new trade
 4. View trading data
 5. View current portfolio
 6. Save trading data
 7. Quit
File not found.
Cannot be blank.
16 trades loaded.

Please choose from the options below
 1. Load trading data
 2. Load current stock prices
 3. Manually enter a new trade
 4. View trading data
 5. View current portfolio
 6. Save trading data
 7. Quit
File not found.
5 current prices loaded.

Please choose from the options below
 1. Load trading data
 2. Load current stock prices
 3. Manually enter a new trade
 4. View trading data
 5. View current portfolio
 6. Save trading data
 7. Quit
ABC
Total units:                      1500
Current value unknown.
BUR
Total units:                      5500
Current value unknown.
FSF
Total units:                      2300
Total value: $                   15042.0
IBN
Total units:           